In [ ]:
import os
os.chdir("..")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
from TravelAndMutate.datamanager import getHeteroAttributes

In [ ]:
filename = "morefocused"
plotdir = f"analysis/{filename}/time_and_space/"
if not os.path.isdir(plotdir):
	os.makedirs(plotdir)
file = h5py.File(f"analysis/{filename}.h5")

attrs = getHeteroAttributes(file)
attrs

In [ ]:
attrs["R0"] = attrs["betas"] / 0.2
attrs

In [ ]:
def plotTInvasion(data, ax):
	subset = data.loc[data["event"] == 0]
	sb.boxplot(data=subset, x="t", y="patch_name", ax=ax)
	ax.set_xlabel("Time of first infection")
	return 
	
def plotTExtinction(data, ax):
	subset = data.loc[data["event"] == -1]
	sb.boxplot(data=subset, x="t", y="patch_name", ax=ax)
	ax.set_xlabel("Time of last infection")
	return
	
def plotNPeaks(data, ax):
	subset = data.loc[data["event"] > 0]
	subset = subset.groupby(["seed","patch_name"], observed=True).count()["t"].reset_index()
	sb.boxplot(data=subset, x="t", y="patch_name", ax=ax)
	ax.set_xlabel("N° peaks")
	return
	
def plotTFirstPeak(data, ax):
	subset = data.loc[data["event"] == 1]
	sb.boxplot(data=subset, x="t", y="patch_name", ax=ax)
	ax.set_xlabel("Time of first peak")
	return
	
def plotTLastPeak(data, ax):
	subset = data.loc[data["event"] > 0]
	subset = subset.sort_values("t")
	subset = subset.groupby(["seed","patch_name"], observed=True).last()
	sb.boxplot(data=subset, x="t", y="patch_name", ax=ax)
	ax.set_xlabel("Time of last peak")
	return
	
def plotTAllPeaks(data, ax):
	subset = data.loc[data["event"] > 0]
	sb.boxplot(data=subset, x="t", y="patch_name", ax=ax)
	ax.set_xlabel("Time of all peaks")
	return

In [ ]:
metrics = {
	"t_first_infection" : plotTInvasion,
	"t_extinction" : plotTExtinction,
	"n_peaks" : plotNPeaks,
	"t_first_peak" : plotTFirstPeak,
	"t_last_peak" : plotTLastPeak,
	"t_all_peaks" : plotTAllPeaks
}
labels = {
	"fitness_p" : r"$P\left(\Delta\varphi^+\right)$",
	"fitness_delta+" : r"$\Delta\varphi^+$",
	"fitness_delta-" : r"$\Delta\varphi^-$",
	"mutation_rate" : r"mean($\eta$)",
	"betas" : r"$\beta$",
	"epsilons" : r"$\varepsilon$",
	"mus" : r"$\mu$"
}

In [ ]:
def plot(file, fixed_params, alongrows, alongcolumns):
	mask = np.ones_like(attrs.index, dtype=bool)
	for key,val in fixed_params.items():
		mask = np.logical_and(mask, attrs[key] == val)
	masked_attrs = attrs.loc[mask].copy().sort_values([alongcolumns,alongrows], ascending=[False,True])
	patch_names = pd.read_csv("inputparams/patchIDs/italy.csv", index_col="Provincia ID")
	nrows = masked_attrs[alongcolumns].unique().shape[0]
	ncols = masked_attrs[alongrows].unique().shape[0]
	figs_and_axss = {metric : plt.subplots(nrows, ncols, figsize=(4*ncols,16*nrows), sharey=True) for metric in metrics.keys()}
	for i,groupname in enumerate(masked_attrs.index):
		if "geography" in file[groupname]:
			data = pd.DataFrame.from_records(file[f"{groupname}/geography"][:])
			data["patch_name"] = pd.Series(patch_names["Capoluogo"].loc[data["loc_id"]].values, dtype="category")
			for metric,plotfunc in metrics.items():
				plotfunc(data, figs_and_axss[metric][1][i//ncols,i%ncols])
	for metric,(fig,axs) in figs_and_axss.items():
		if not os.path.isdir(plotdir+metric):
			os.makedirs(plotdir+metric)
		for i in range(len(axs)):
			ax = axs[i//ncols,i%ncols]
			label = [
				r"$P\left(\Delta\varphi^+\right)$"+f" = {round(masked_attrs.iloc[i]['fitness_p'], 2)}",
				r"mean($\eta$)"+f" = {masked_attrs.iloc[i]['mutation_rate']}"
			]
			ax.set_title("\t".join(label))
		title = "\n".join([f"{labels[key]} = {str(val)}" for key,val in fixed_params.items()])
		fig.suptitle(title, y=1)
		fig.tight_layout()
		filename = plotdir+metric + "/" + "-".join(masked_attrs.index) + ".png"
		fig.savefig(filename)
		plt.close(fig)
	return

In [ ]:
for deltap in attrs["fitness_delta+"].unique():
	for deltam in attrs["fitness_delta-"].unique():
		for beta in attrs["betas"].unique():
			fixed = {"fitness_delta-":deltam, "fitness_delta+":deltap, "betas":beta}
			plot(file, fixed, "mutation_rate", "fitness_p")

In [ ]:
file.close()